In [319]:
#import required libraries
import numpy as np
import pandas as pd
import math

from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from datetime import datetime

In [135]:
np.random.seed(42)

# LOAD DATA

In [265]:
df_train = pd.read_csv('train_users_2.csv')
df_train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [266]:
df_age_gender = pd.read_csv('age_gender_bkts.csv')
df_age_gender.head()

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0
3,85-89,AU,male,118.0,2015.0
4,80-84,AU,male,199.0,2015.0


In [267]:
df_session = pd.read_csv('sessions.csv')
df_session.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [268]:
df_countries = pd.read_csv('countries.csv')
df_countries.head()

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06


# HANDLING NULL VALUES IN DATA

In [269]:
#Function to get the features containing null values
def get_null_features(df):
    na_count_df=df.isnull().sum()
    return na_count_df[na_count_df>0].index.values

In [270]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [271]:
for i in [df_train,df_age_gender,df_session,df_countries]:
    print("**************************************************************")
    if i.isnull().values.any():
        print("{} has null values in follwing features: {}".format(get_df_name(i),get_null_features(i)))
    else:
        print("{} does not have any null values".format(get_df_name(i)))
      

**************************************************************
df_train has null values in follwing features: ['date_first_booking' 'age' 'first_affiliate_tracked']
**************************************************************
df_age_gender does not have any null values
**************************************************************
df_session has null values in follwing features: ['user_id' 'action' 'action_type' 'action_detail' 'device_type'
 'secs_elapsed']
**************************************************************
df_countries does not have any null values


In [272]:
df_train['first_affiliate_tracked']=df_train['first_affiliate_tracked'].fillna('untracked')

In [273]:
df_train = df_train[pd.notnull(df_train['date_first_booking'])]

In [274]:
df_train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
5,osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US


In [275]:
df_session = df_session[pd.notnull(df_session['user_id'])]

In [276]:
df_session[pd.isnull(df_session['user_id'])]

,user_id,action,action_type,action_detail,device_type,secs_elapsed


# CREATING BUCKETS FOR AGE IN df_train

In [277]:
df=df_train

In [278]:
df_train=df

In [279]:
df_train['age1900']=df_train['age'].apply(lambda x: (2015-x) if (x>=1900 and x<=2000) else x)

In [280]:
df_train['age_binned'] = pd.cut(df_train['age1900'], [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,150,2000,2015], include_lowest=True)

In [281]:
df_train['age_binned']=df_train['age_binned'].apply(lambda x: (str(x.left)+'-'+str(x.right)))

In [282]:
df_train.drop(['age','age1900'],inplace=True,axis=1)

In [283]:
df_train['age_binned']=df_train['age_binned'].fillna('2000.0-2015.0')

In [284]:
df_train['age_binned']=df_train['age_binned'].apply(lambda x: 'incorrect age' if (x in ['100.0-150.0','2000.0-2015.0']) else x)

# Merging user and sessions data

In [285]:
df_train_session=pd.merge(df_train, df_session, left_on='id', right_on='user_id', how='inner')

In [286]:
df_train_session.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,first_device_type,first_browser,country_destination,age_binned,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,2014-01-01,20140101000936,2014-01-04,MALE,basic,0,en,sem-non-brand,google,...,Windows Desktop,Chrome,other,60.0-65.0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,2014-01-01,20140101000936,2014-01-04,MALE,basic,0,en,sem-non-brand,google,...,Windows Desktop,Chrome,other,60.0-65.0,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,2014-01-01,20140101000936,2014-01-04,MALE,basic,0,en,sem-non-brand,google,...,Windows Desktop,Chrome,other,60.0-65.0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,2014-01-01,20140101000936,2014-01-04,MALE,basic,0,en,sem-non-brand,google,...,Windows Desktop,Chrome,other,60.0-65.0,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,2014-01-01,20140101000936,2014-01-04,MALE,basic,0,en,sem-non-brand,google,...,Windows Desktop,Chrome,other,60.0-65.0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [287]:
len(df_train_session)

438757

# Handle remaining missing values

In [288]:
df_train_session['action']=df_train_session['action'].fillna('msg')
df_train_session[df_train_session['action']=='msg']

,id,date_account_created,timestamp_first_active,date_first_booking,gender,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,first_device_type,first_browser,country_destination,age_binned,user_id,action,action_type,action_detail,device_type,secs_elapsed
45,d1mm9tcy42,2014-01-01,20140101000936,2014-01-04,MALE,basic,0,en,sem-non-brand,google,...,Windows Desktop,Chrome,other,60.0-65.0,d1mm9tcy42,msg,message_post,message_post,Windows Desktop,NaN
137,xwxei6hdk4,2014-01-01,20140101002742,2014-01-07,FEMALE,facebook,0,en,seo,google,...,iPad,Mobile Safari,US,30.0-35.0,xwxei6hdk4,msg,message_post,message_post,iPad Tablet,NaN
140,xwxei6hdk4,2014-01-01,20140101002742,2014-01-07,FEMALE,facebook,0,en,seo,google,...,iPad,Mobile Safari,US,30.0-35.0,xwxei6hdk4,msg,message_post,message_post,iPad Tablet,0.0
156,ro2stddszp,2014-01-01,20140101005503,2014-12-04,-unknown-,basic,0,en,sem-brand,google,...,Mac Desktop,Safari,other,15.0-20.0,ro2stddszp,msg,message_post,message_post,Mac Desktop,7192.0
158,ro2stddszp,2014-01-01,20140101005503,2014-12-04,-unknown-,basic,0,en,sem-brand,google,...,Mac Desktop,Safari,other,15.0-20.0,ro2stddszp,msg,message_post,message_post,Mac Desktop,NaN
176,ro2stddszp,2014-01-01,20140101005503,2014-12-04,-unknown-,basic,0,en,sem-brand,google,...,Mac Desktop,Safari,other,15.0-20.0,ro2stddszp,msg,message_post,message_post,Mac Desktop,115762.0
186,awiurksqr3,2014-01-01,20140101010113,2014-01-02,FEMALE,facebook,0,en,direct,direct,...,iPad,Mobile Safari,US,30.0-35.0,awiurksqr3,msg,message_post,message_post,iPad Tablet,NaN
713,jrqykh9y8x,2014-01-01,20140101011919,2014-04-19,FEMALE,facebook,0,zh,seo,google,...,Mac Desktop,Chrome,FR,25.0-30.0,jrqykh9y8x,msg,message_post,message_post,Mac Desktop,1362.0
742,jrqykh9y8x,2014-01-01,20140101011919,2014-04-19,FEMALE,facebook,0,zh,seo,google,...,Mac Desktop,Chrome,FR,25.0-30.0,jrqykh9y8x,msg,message_post,message_post,Mac Desktop,1732.0
809,jrqykh9y8x,2014-01-01,20140101011919,2014-04-19,FEMALE,facebook,0,zh,seo,google,...,Mac Desktop,Chrome,FR,25.0-30.0,jrqykh9y8x,msg,message_post,message_post,Mac Desktop,1831.0


In [289]:
df_train_session['action_type']=df_train_session['action_type'].fillna('atna')

In [290]:
df_train_session['action_detail']=df_train_session['action_detail'].fillna('adna')

In [291]:
df_train_session['device_type']=df_train_session['device_type'].fillna('dtna')

In [292]:
df_train_session['secs_elapsed']=df_train_session['secs_elapsed'].fillna(0)

In [293]:
df_train_session.isnull().sum().any()

False

# Convert dates to Julian date format

In [294]:
df_train_session.drop('timestamp_first_active',inplace=True,axis=1)

In [295]:
type(df_train_session['date_account_created'][0])

str

In [296]:
type(df_train_session['date_first_booking'][0])

str

In [298]:
df_train_session['date_account_created']= pd.to_datetime(df_train_session['date_account_created'])
df_train_session['date_first_booking']= pd.to_datetime(df_train_session['date_first_booking'])

In [300]:
df_train_session['date_account_created']=df_train_session['date_account_created'].apply(lambda x: x.to_julian_date())
df_train_session['date_first_booking']=df_train_session['date_first_booking'].apply(lambda x: x.to_julian_date())

# Encoding categorical features

In [304]:
#Convert categorical features to numerical form
def encoding(df,categorical_features):
    
    df_cat = df[categorical_features]
    #By defaault, convert all the categories to numerical forms using Label encoder
    le = {}#create a dictionary to store encodings of different categorical features
    for i in range(len(df_cat.columns.tolist())):
        le[i] = LabelEncoder()
        df_cat.iloc[:,i] = le[i].fit_transform(df_cat.iloc[:,i].astype(str))

    return df_cat

In [310]:
df_train_session['signup_flow']=df_train_session['signup_flow'].astype('str')

In [315]:
cols = df_train_session.columns
num_cols = df_train_session._get_numeric_data().columns
categorical_features=list(set(cols) - set(num_cols))

encoded_cat_data  = encoding(df_train_session,categorical_features)
#concatenate encoded categorical features and continuous features
encoded_data          = pd.concat([encoded_cat_data,df_train_session[num_cols]],axis=1)
encoded_data.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,first_affiliate_tracked,signup_app,first_browser,affiliate_provider,signup_method,age_binned,first_device_type,id,action,affiliate_channel,...,action_detail,device_type,language,country_destination,user_id,action_type,gender,date_account_created,date_first_booking,secs_elapsed
0,2,2,5,6,0,10,6,2148,107,6,...,6,9,4,10,2148,1,2,2456658.5,2456661.5,319.0
1,2,2,5,6,0,10,6,2148,175,6,...,93,9,4,10,2148,3,2,2456658.5,2456661.5,67753.0
2,2,2,5,6,0,10,6,2148,107,6,...,6,9,4,10,2148,1,2,2456658.5,2456661.5,301.0
3,2,2,5,6,0,10,6,2148,175,6,...,93,9,4,10,2148,3,2,2456658.5,2456661.5,22141.0
4,2,2,5,6,0,10,6,2148,107,6,...,6,9,4,10,2148,1,2,2456658.5,2456661.5,435.0


# Scaling

In [317]:
def scale_data(df):  

    scaler = MinMaxScaler()#scales based on minimum and maximum values in dataset

    return pd.DataFrame(scaler.fit_transform(df),columns=df.columns,index=df.index)

In [321]:
scaled_data=encoded_data
scaled_data.loc[:, df.columns != 'country_destination'] = scale_data(encoded_data.loc[:, df.columns != 'country_destination'] )

In [322]:
scaled_data.head()

,first_affiliate_tracked,signup_app,first_browser,affiliate_provider,signup_method,age_binned,first_device_type,id,action,affiliate_channel,...,action_detail,device_type,language,country_destination,user_id,action_type,gender,date_account_created,date_first_booking,secs_elapsed
0,0.4,0.666667,0.263158,0.5,0.0,0.555556,0.75,0.371819,0.486364,0.857143,...,0.060606,0.692308,0.25,10,0.371819,1,2,2456658.5,2456661.5,319.0
1,0.4,0.666667,0.263158,0.5,0.0,0.555556,0.75,0.371819,0.795455,0.857143,...,0.939394,0.692308,0.25,10,0.371819,3,2,2456658.5,2456661.5,67753.0
2,0.4,0.666667,0.263158,0.5,0.0,0.555556,0.75,0.371819,0.486364,0.857143,...,0.060606,0.692308,0.25,10,0.371819,1,2,2456658.5,2456661.5,301.0
3,0.4,0.666667,0.263158,0.5,0.0,0.555556,0.75,0.371819,0.795455,0.857143,...,0.939394,0.692308,0.25,10,0.371819,3,2,2456658.5,2456661.5,22141.0
4,0.4,0.666667,0.263158,0.5,0.0,0.555556,0.75,0.371819,0.486364,0.857143,...,0.060606,0.692308,0.25,10,0.371819,1,2,2456658.5,2456661.5,435.0
